## Import necessary modules

In [7]:
import json
from spacy import displacy
from IPython.core.display import display, HTML
import os
import webbrowser

In [8]:
def read_json(filename):
    filepath = "./debug/" + filename + ".json"
    
    with open(filepath, 'r') as file:
        contents = json.load(file)
    
    return contents

def write_json(filename, content):
    filepath = "./debug/" + filename + ".json"
    
    with open(filepath, 'w') as file:
        json.dump(content, file)
        
write_json("unrecognized", [])

In [9]:
def create_debug(text_ID, text, text_PHIs, unrecognized, altered_text, PHI_to_tag):
    
    #create a list of all unrecognized PHI tags, stored in a json file
    if read_json("unrecognized") == []:
            write_json("unrecognized", unrecognized)
    else:
        contents = read_json("unrecognized")
        contents.append(unrecognized)
        write_json("unrecognized", contents)
    
    #setting custom colors for each type so that it is easier to visualize (using color hex codes)
    displacy_colors = read_json("displacy_colors")
    
    #create annotated HTML versions of the original texts, color coded by PHI type, and save them to a json file
    data = [{
        "text": text,
        "ents": [],
        "title": "Text " + str(text_ID) + " (Original)"
    }]
    
    for PHI in text_PHIs:
        data[0]["ents"].append({
            "start": PHI["location"][0],
            "end": PHI["location"][1],
            "label": PHI["type"]
        })
    
    original_html = displacy.render(data, style="ent", jupyter=False, manual=True, options={"colors":displacy_colors})
    
    if read_json("original_htmls") == []:
        write_json("original_htmls", [original_html])
    else:
        contents = read_json("original_htmls")
        contents.append(original_html)
        write_json("original_htmls", contents)
    
    #create annotated HTML versions of the altered texts, color coded by PHI type, and save them to a json file
    data = [{
        "text": altered_text,
        "ents": [],
        "title": "Text " + str(text_ID) + " (Altered)"
    }]
    
    for PHI in PHI_to_tag:
        data[0]["ents"].append({
            "start": PHI["start"],
            "end": PHI["end"],
            "label": PHI["type"][0]
        })
    
    altered_html = displacy.render(data, style="ent", jupyter=False, manual=True, options={"colors":displacy_colors})
    
    if read_json("altered_htmls") == []:
        write_json("altered_htmls", [altered_html])
    else:
        contents = read_json("altered_htmls")
        contents.append(altered_html)
        write_json("altered_htmls", contents)
        
    return None

In [10]:
def render_displacy_html(html_ID, html_type, jupyter_render=False):
    html_list = read_json(html_type + "_htmls")
    html_to_export = html_list[html_ID]
    
    if jupyter_render:
        display(HTML(html_to_export))
        pass
    else:
        #save the html to a file
        with open("./debug/debug.html", 'w') as file:
            file.write(html_to_export)
            file.close()
        
        #find the absolute filepath of the created file
        full_filepath = os.path.abspath('./debug/debug.html')
        
        #open the file in the default web browser
        webbrowser.open(full_filepath, new=2) #new=2 tells the webbrowser library to open the file in a new tab

In [11]:
def reset_jsons():
    write_json("unrecognized", [])
    write_json("original_htmls", [])
    write_json("altered_htmls", [])